In [1]:
import ccxt
import datetime
import time

In [15]:
exchange = ccxt.binance()

# Set the start and end date for the data
start_date = datetime.datetime(2017, 8, 17,12,0,0).astimezone(datetime.timezone.utc)
end_date = datetime.datetime(2024, 9, 1,8,0,0).astimezone(datetime.timezone.utc)

period = '1h'
symbol = 'BTC/USDT:USDT'

current_ts = int(start_date.timestamp()* 1000)
end_ts = int(end_date.timestamp()* 1000)


all_data = []

while current_ts < end_ts:
    
    period_second = {
        '1m': 60,
        '3m': 180,
        '5m': 300,
        '15m': 900,
        '30m': 1800,
        '1h': 3600,
        '2h': 7200,
        '4h': 14400,
        '6h': 21600,
        '8h': 28800,
        '12h': 43200,
        '1d': 86400,
        '3d': 259200,
        '1w': 604800,
        '1M': 2592000
    }
    
    data = exchange.fetch_ohlcv (symbol,period, since=current_ts) 
    current_ts = int(data[-1][0] + period_second[period] * 1000)
    
    for i in data:
        all_data.append(i)
    time.sleep(exchange.rateLimit / 1000) # time.sleep wants seconds    


In [16]:
import pandas as pd
df = pd.DataFrame(all_data, columns=['timestamp', 'Open', 'High', 'Low', 'Close', 'Volume'])

In [17]:
df.to_csv('BTCUSDT_1h.csv', index=False)

In [15]:
# get funding rate history

all_data = []

start_ts = 1568102400000

while start_ts < end_ts:
    data = exchange.fetchFundingRateHistory(symbol='ETH/USDT:USDT',since=start_ts)
    start_ts = int(data[-1]['timestamp'])
    for i in data:
        all_data.append(i)
    time.sleep(exchange.rateLimit / 1000)
    


In [16]:
df_funding_rate = pd.DataFrame(all_data,columns = ['timestamp','fundingRate'])

In [17]:
df_funding_rate

,timestamp,fundingRate
0,1574841600000,0.000100
1,1574870400000,0.000100
2,1574899200000,0.000100
3,1574928000000,0.000014
4,1574956800000,-0.000106
...,...,...
5295,1725840000000,0.000008
5296,1725868800000,0.000031
5297,1725897600001,-0.000012
5298,1725926400000,0.000011


In [61]:
# rolling , - the previous value
df_funding_rate['time_diff'] = df_funding_rate['timestamp'].diff()

In [18]:
df_funding_rate.drop_duplicates(subset= ['timestamp'],inplace=True)

In [19]:
df_funding_rate['time_diff'] = df_funding_rate ['timestamp'].diff()

In [20]:
df_funding_rate['time_diff'].describe()

count    5.247000e+03
mean     2.880000e+07
std      6.895163e+00
min      2.879996e+07
25%      2.880000e+07
50%      2.880000e+07
75%      2.880000e+07
max      2.880005e+07
Name: time_diff, dtype: float64

In [21]:
df_funding_rate[['timestamp','fundingRate']].to_csv('ETHUSDT_funding_rate.csv',index=False)